# Standalone App

Our scale works great, except that it is still tethered to our development platform. Let's convert it into a standalone application.

On power on, MicroPython looks for a file called `main.py` and executes it if found. Hence all we have to do is save our code to `main.py` and upload this file.

In [1]:
%cd $IOT/iot49.org/docs/projects/balance
%connect balance

cwd = /home/iot/iot49.org/docs/projects/balance
Connected to balance @ serial:///dev/ttyUSB0


In [3]:
%%writefile lib/main.py

import ble_uart_peripheral
import bluetooth

from machine import Pin, I2C
from ssd1306 import SSD1306_I2C
from scale import Scale
from button import Button

# configure the Bluetooth UART
ble = bluetooth.BLE()
uart = ble_uart_peripheral.BLEUART(ble)

# Scale & Display
i2c = I2C(0, scl=Pin(22), sda=Pin(23))

oled_width = 128
oled_height = 32
oled = SSD1306_I2C(oled_width, oled_height, i2c)

scale = Scale()
tare_button = Button(15, scale.tare)

last_weight = 500
while True:
    weight = scale.measure()
    if abs(weight-last_weight) > 3:
        # send via Bluetooth
        uart.write("{:8.0f} gram\n".format(weight))
        # show on display
        oled.fill(0)
        oled.text("{:8.0f} gram".format(weight), 0, 12)
        oled.show()
        last_weight = weight

Writing lib/main.py


Modify the device configuration to include `main.py`.

In [2]:
!cat devices/balance.yaml

balance:
    uid: 24:0a:c4:12:87:7c
    resources:
        - lib
        - main.py: /


In [1]:
%rsync

Directories match


That's it! Plug the device into a USB power source or just power cycle and continue using the Raspberry Pi as power source. The app will start running-check the display and Bluetooth.